In [113]:
import pandas as pd
import numpy as np
from numpy import nan


In [114]:
df = pd.read_csv("jobs14:03:24.csv")

In [115]:
df.head()

,Unnamed: 0,title,company,salary,location,company_rating,description
0,0,Junior Data Scientist (NYC or Remote),ViacomCBS,NaN,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...
1,1,"Associate Data Scientist, Content Analytics",Spotify,NaN,Remote in Oregon+12 locations,4.3,"Data, Research & Insights\nData Science\nSpoti..."
2,2,"Data Scientist, TikTok Creation and Consumption",TikTok,NaN,"San Francisco, CA+34 locations",3.5,TikTok is the leading destination for short-fo...
3,3,"Data Scientist, Game Analytics & Strategy",the NBA,NaN,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and..."
4,4,Data Scientist and Analytics Specialist,Delta,NaN,"Atlanta, GA+4 locations",4.2,"United States, Georgia, Atlanta\nProduct Manag..."


In [116]:
df.shape

(450, 7)

We will start to work from salary column. Goal here is to have 
1) mean value for the salary if they have a salary estimation
2) or leave it as NaN

First Let's figure out
- How many NaN value we have
- How many value that we can use
- structure of non NaN values
- extract only numbers

In [117]:
print('we have', df['salary'].isna().sum(), 'Nan value for the salary column')
print('that indicates that we have only 56 values to use')

we have 394 Nan value for the salary column
that indicates that we have only 56 values to use


In [118]:
salary = df['salary']
salary_notnull = salary[salary.notnull()]
print(salary_notnull)


11           $110,000 - $140,000 a year
12            $72,272 - $176,300 a year
17            $92,700 - $160,000 a year
21                      $110,500 a year
23            $83,690 - $160,000 a year
26            $70,400 - $117,400 a year
29            $76,000 - $148,000 a year
33            $83,690 - $160,000 a year
35           $120,000 - $223,000 a year
36            $72,272 - $176,300 a year
39     Estimated $47.7K – $60.3K a year
41            $90,000 - $150,000 a year
46            $90,000 - $150,000 a year
55            $86,389 - $116,788 a year
57            $70,400 - $117,400 a year
58                       $95,000 a year
69            $65,072 - $117,129 a year
71           $125,000 - $175,000 a year
72            $90,000 - $150,000 a year
75           $118,248 - $127,208 a year
93                       $98,000 a year
96             $4,372 - $10,792 a month
104           $90,000 - $161,000 a year
112           $90,000 - $161,000 a year
117           $96,400 - $144,600 a year


Now let's extract numbers only from the salary columns

In [119]:
def str_contains_n(str):
    num_str = "0123456789"
    word_list = []
    words = str.split(" ")
    for word in words:
        word_has_n = False
        for char in word:
            if char in num_str:
                word_has_n = True
        if word_has_n:
            word_list.append(word)
    result = '-'.join(word_list)
    return result

In [120]:
df['k_unit'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'k' in x.lower() else 0)
df['yearly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a year' in x.lower() else 0)
df['monthly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a month' in x.lower() else 0)
df['hourly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'an hour' in x.lower() else 0)

salary = salary.apply(lambda x: str_contains_n(x) if isinstance(x, str) else None)
salary = salary.apply(lambda x: x.lower().replace("$","").replace("k","").replace(",","") if x != None else None)
salary = salary.apply(lambda x: x if isinstance(x,str) else 0)

df['salary_edit'] = salary

df['min_salary'] = salary.apply(lambda x: float(x.split('-')[0]) if isinstance(x,str) else 0)
df['max_salary'] = salary.apply(lambda x: float(x.split('-')[1]) if isinstance(x,str) and len(x.split('-')) >1 else(float(x.split('-')[0]) if isinstance(x,str) else 0))
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

# df[df['salary_edit'] != 0]
# df[df['k_unit'] == 1].min_salary * 1000
df.loc[df['k_unit'] == 1,['min_salary']] = df[df['k_unit'] == 1].min_salary * 1000
df.loc[df['k_unit'] == 1,['max_salary']] = df[df['k_unit'] == 1].max_salary * 1000
df.loc[df['k_unit'] == 1,['avg_salary']] = df[df['k_unit'] == 1].avg_salary * 1000
df.loc[df['k_unit'] == 1]


# df.at[df['k_unit'] == 1, 'Product_Name'] = df[df['k_unit'] == 1].min_salary * 1000


,Unnamed: 0,title,company,salary,location,company_rating,description,k_unit,yearly,monthly,hourly,salary_edit,min_salary,max_salary,avg_salary
39,39,Data Scientist / Biostatistician - Sumner Lab,UNC-Chapel Hill,Estimated $47.7K – $60.3K a year,"Chapel Hill, NC 27599+1 location",4.1,A full-time temporary Data Scientist/Biostatis...,1,1,0,0,47.7-60.3,47700.0,60300.0,54000.0
228,228,Junior data scientist / bioinformatics,Data Tecnica,Estimated $117K – $149K a year,"Remote in Glen Echo, MD",NaN,NaN,1,1,0,0,117-149,117000.0,149000.0,133000.0
341,341,Decision Scientist,PayPal,Estimated $63.7K – $80.7K a year,"New York, NY 10014 (West Village area)+2 locat...",3.9,"Who we are: At PayPal (NASDAQ: PYPL), we belie...",1,1,0,0,63.7-80.7,63700.0,80700.0,72200.0
